In [16]:
# Confirm if this is my desktop
import IPython
import sklearn, numpy as np, scipy
from ast import literal_eval
CURR_IPYTHON_VERSION = IPython.__version__
PYTHON_INPT_VERSION = literal_eval(IPython.sys_info())['sys_executable'] + " --version"
SKVERSION = sklearn.__version__; SCVERSION = scipy.__version__; NPVERSION = np.__version__
!echo "At host \"$(hostname)\" with $(nproc) cores. Current Dir - $(pwd)"; 
!echo -n "sklearn $(python -c 'import sklearn; print sklearn.__version__') "
!echo -n "in branch \"$(git --git-dir $HOME/raghav/scikit-learn/.git rev-parse --abbrev-ref HEAD)\", "
!echo -n "(last commit \"$(git --git-dir $HOME/raghav/scikit-learn/.git log --pretty=format:'%h' -n 1)\")"
!echo -e -n "- np v$NPVERSION - scipy v$SCVERSION\nRunning on IPython v$CURR_IPYTHON_VERSION; "
!echo -n "`$PYTHON_INPT_VERSION`"
!echo "@ /tsi/doctorants/raghav/anaconda/anaconda3"

At host "tsilinuxd98" with 8 cores. Current Dir - /cal/homes/vrajagopalan/raghav/miss_val_bench
sklearn 0.18.dev0 in branch "missing_values_rf", (last commit "359cc5d")- np v1.10.4 - scipy v0.17.0
Running on IPython v4.0.3; Python 2.7.11 :: Anaconda 2.5.0 (64-bit)
@ /tsi/doctorants/raghav/anaconda/anaconda3


In [4]:
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
plt.rcParams['figure.figsize'][:] = [10, 10]

In [2]:
print 5

5


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

baseline_score = np.load('baseline_score.npy')
missing_fraction_range = np.load('missing_fraction_range.npy')
scores_missing = np.load('scores_missing.npy')
scores_impute = np.load('scores_impute.npy')

In [ ]:
plt.close('all')
plt.plot(missing_fraction_range, seconds_missing, '.--', color='r', label='RF MV enabled')
plt.plot(missing_fraction_range, seconds_impute, '.--', color='b', label='RF+imputer')
plt.axhline(35, label='RF w/No missing', color='k')
#for sample_pt in missing_fraction_range:
#    plt.axvline(sample_pt, linestyle='--', color='g')
plt.xlabel('Missing fraction')
plt.ylabel('Time taken for cross_val_score using 3 iterations of StratifiedShuffleSplit in seconds')
plt.legend(loc='best')
plt.show()

In [ ]:
from sklearn.datasets import fetch_mldata

adult = fetch_mldata('yeast')

In [7]:
cat_feats = np.load('cat_feats.npy').tolist()
feat_names = np.load('feat_names.npy').tolist()
data = np.load('data.npy')
target = np.load('target.npy')

In [8]:
data.shape, data.dtype

((48842, 12), dtype('float64'))

In [9]:
np.mean(np.isnan(data)) * 100

1.1030465582899962

In [31]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import Imputer
from sklearn.model_selection import cross_val_score
import time

In [36]:
rf_missing = RandomForestClassifier(n_estimators=100,
                                    missing_values='NaN',
                                    n_jobs=-1)

rf_impute = Pipeline([('imp', Imputer(strategy='mean')), 
                      ('rf', RandomForestClassifier(n_estimators=100,
                                                    n_jobs=-1))])


rf_impute2 = Pipeline([('imp', Imputer(strategy='mean')), 
                      ('rf', RandomForestClassifier(n_estimators=100,
                                                    n_jobs=-1))])

cv = StratifiedKFold(n_folds=15)


print "The cross val score with RF (MV) (computed in ", 
t = time.time()
cv_rf_missing = cross_val_score(rf_missing, X=data, y=target, cv=cv)
t -= time.time()
print "%0.2f seconds)." % abs(t), 
print "%0.4f" % np.mean(cv_rf_missing)

print "The cross val score with RF+Imp(Median) (computed in ", 
t = time.time()
cv_rf_imp_median = cross_val_score(rf_impute, X=data, y=target, cv=cv)
t -= time.time()
print "%0.2f seconds)." % abs(t), 
print "%0.4f" % np.mean(cv_rf_imp_median)

print "The cross val score with RF+Imp(Mean) (computed in ", 
t = time.time()
cv_rf_imp_mean = cross_val_score(rf_impute2, X=data, y=target, cv=cv)
t -= time.time()
print "%0.2f seconds)." % abs(t), 
print "%0.4f" % np.mean(cv_rf_imp_mean)

The cross val score with RF (MV) (computed in  57.51 seconds). 0.8525
The cross val score with RF+Imp(Median) (computed in  56.93 seconds). 0.8488
The cross val score with RF+Imp(Mean) (computed in  59.11 seconds). 0.8486


In [37]:

data.shape

(48842, 12)

In [42]:
import numpy as np

from sklearn.cluster import KMeans

km = KMeans(n_clusters=2)
X = np.array([[1], [2], [3], [10], [12], [13]])
km.fit(X)

print X.shape

print km.labels_

(6, 1)
[1 1 1 0 0 0]


In [46]:
km = KMeans(n_clusters=1)
km.fit(np.ravel(X))

/home/rvraghav93/.local/lib/python2.7/site-packages/sklearn/utils/validation.py:407: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


KMeans(copy_x=True, init='k-means++', max_iter=300, n_clusters=1, n_init=10,
    n_jobs=1, precompute_distances='auto', random_state=None, tol=0.0001,
    verbose=0)

In [47]:
km.labels_

array([0], dtype=int32)

In [50]:
from sklearn.tree import export_graphviz

In [51]:
export_graphviz?

In [3]:
k

The shape of the dataset is (581012, 54)
The number of trees for this benchmarking is 100
Score with the entire dataset = 0.95
Score RF with the 1.82173199202 % missing = 0.94
Score RF+Imp. with the 1.82173199202 % missing = 0.95
Score RF with the 3.55273467929 % missing = 0.96
Score RF+Imp. with the 3.55273467929 % missing = 0.96
Score RF with the 5.19537302857 % missing = 0.96
Score RF+Imp. with the 5.19537302857 % missing = 0.96
Score RF with the 6.75907503408 % missing = 0.97
Score RF+Imp. with the 6.75907503408 % missing = 0.97
Score RF with the 8.24469488869 % missing = 0.97
Score RF+Imp. with the 8.24469488869 % missing = 0.97
Score RF with the 9.65472823791 % missing = 0.97
Score RF+Imp. with the 9.65472823791 % missing = 0.97
Score RF with the 10.9961424906 % missing = 0.97
Score RF+Imp. with the 10.9961424906 % missing = 0.98
Score RF with the 12.2712707406 % missing = 0.98
Score RF+Imp. with the 12.2712707406 % missing = 0.98
Score RF with the 13.4793894739 % missing = 0.98
